The following pip installs are necessary in whatever environment you choose to work in:

youtube_transcript_api
googleapiclient
pandas
os
pickle
langchain
openai
chromadb
langchain-openai



Initialise and Set up Google API Key

In [1]:
import pickle
import os
import pandas as pd
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi


f = open("keys/google_key.txt", "r")
key = f.readlines()[0]
f.close()

youtube = build('youtube', 'v3', developerKey=key)




User Inputs (prompt)

In [2]:
search_terms = "Apple Watch Series 9 Review"    # insert prompt -> "Brand Product Model"
max_result = 10                         # No. of results (1-50)


Define containers to store info

In [3]:
vid_id = []             	  # video id
vid_page = []       		    # video links (https...)
vid_title = []              # video title
num_comments = []           # official number of comments
load_error = 0              # error counter
can_load_title = []         # temp. list for storing title w/o loading error
can_load_page = []          # temp. list for storing links w/o loading error
num_page = []               # comment_response page number
page_title = []             # comment_response video title
comment_resp = []           # comment_response
comment_list = []           # temp. list for storing comments
comment_data = []           # comments & replies from comment_response
all_count = 0               # total number of comments


Search for Video IDs based on User Inputs

In [4]:
print("Search for Videos IDs...")
request = youtube.search().list(
    q=search_terms,
    maxResults=max_result,
    part="id",
    type="video"
    )
search_response = request.execute()
print(search_response)


Search for Videos IDs...
{'kind': 'youtube#searchListResponse', 'etag': 'Dk8xkh5a8VHCaqOXCGvh_cJ0Yqk', 'nextPageToken': 'CAoQAA', 'regionCode': 'SG', 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 10}, 'items': [{'kind': 'youtube#searchResult', 'etag': 'dTiKHulGWhMEO_U2DKUeDTeFRm4', 'id': {'kind': 'youtube#video', 'videoId': '5gkiXZ_DjOw'}}, {'kind': 'youtube#searchResult', 'etag': 'ffJYKiKj8QQzNUP8N4_sWa6AtpQ', 'id': {'kind': 'youtube#video', 'videoId': '44FEs8jAN9Y'}}, {'kind': 'youtube#searchResult', 'etag': 'U6oCnPmsIrcFAvecq7pnrKIlb0s', 'id': {'kind': 'youtube#video', 'videoId': 'wM16xWaJHbU'}}, {'kind': 'youtube#searchResult', 'etag': 'bQwrEsWr8pBj48HocXdWwZrxUK8', 'id': {'kind': 'youtube#video', 'videoId': 'oNCs4C2SMjo'}}, {'kind': 'youtube#searchResult', 'etag': 'oQrd6-MZ_Kvk9oOKOryZYaiZAZ4', 'id': {'kind': 'youtube#video', 'videoId': 'Oh9Q6zUqng4'}}, {'kind': 'youtube#searchResult', 'etag': '2et133fNlGjEfRdh0mx6UBtVQx8', 'id': {'kind': 'youtube#video', 'videoId': '5Vy

Create a list of Video IDs and a corresponding list of weblinks

In [6]:
print("Videos found...")
for i in range(max_result):
    videoId = search_response['items'][i]['id']['videoId']
    print(videoId)
    vid_id.append(videoId)                          # a list of Video IDs
    page = "https://www.youtube.com/watch?v=" + videoId
    print(page)
    print()
    vid_page.append(page)                           # a list of Video links
print("\nThere are", len(vid_page), "videos.")



Videos found...
5gkiXZ_DjOw
https://www.youtube.com/watch?v=5gkiXZ_DjOw

44FEs8jAN9Y
https://www.youtube.com/watch?v=44FEs8jAN9Y

wM16xWaJHbU
https://www.youtube.com/watch?v=wM16xWaJHbU

oNCs4C2SMjo
https://www.youtube.com/watch?v=oNCs4C2SMjo

Oh9Q6zUqng4
https://www.youtube.com/watch?v=Oh9Q6zUqng4

5VyLYPyqJvE
https://www.youtube.com/watch?v=5VyLYPyqJvE

J6hCaO7n35o
https://www.youtube.com/watch?v=J6hCaO7n35o

FcF0F4lYi_U
https://www.youtube.com/watch?v=FcF0F4lYi_U

4igZ3DEhrk4
https://www.youtube.com/watch?v=4igZ3DEhrk4

GHYg8cmUiKM
https://www.youtube.com/watch?v=GHYg8cmUiKM


There are 20 videos.


Use the list of Video IDs to get video data

In [10]:
print("Get video data...")
for i in range(len(vid_id)):
    request = youtube.videos().list(
        part="snippet, statistics",
        id=vid_id[i]
        )
    video_response = request.execute()
    print(video_response)

    title = video_response['items'][0]['snippet']['title']
    vid_title.append(title)
    try:                        # use try/except as some videos might not load
        comment_count = video_response['items'][0]['statistics']['commentCount']
        print("Video", i + 1, "-", title, "-- Comment count: ", comment_count)
        print()
        num_comments.append(comment_count)
    except:
        print("Video", i + 1, "-", title, "-- Comments are turned off")
        print()
        num_comments.append(0)


Get video data...
{'kind': 'youtube#videoListResponse', 'etag': 'aUrW7R_1oNBvfXVOLImH7S6Oygs', 'items': [{'kind': 'youtube#video', 'etag': 'N2jx4v2an5iiw3JsWDWdSWbBF68', 'id': '5gkiXZ_DjOw', 'snippet': {'publishedAt': '2023-09-20T13:00:07Z', 'channelId': 'UCOmcA3f_RrH6b9NmcNa4tdg', 'title': 'Apple Watch Series 9 Review: Slightly Smarter and Faster', 'description': "The Series 9’s new Double Tap feature and faster Siri responses make the watch feel more convenient. But otherwise, it’s largely the same as the Series 8.\n\nFind the Apple Watch Series 9 here:\nApple Watch Series 9 -  https://cnet.co/3Zr9MP4\n*Cnet may get a commission for these offers\n\n0:00 What's New\n1:35 Double Tap\n2:55 On-Device Siri\n5:00 Find My Phone\n6:00 Battery, Health and Other Features\n6:53 Final Thoughts\n\nSubscribe to CNET: https://www.youtube.com/user/CNETTV\nNever miss a deal again! See CNET’s browser extension 👉 https://bit.ly/3lO7sOU\nCheck out CNET’s Amazon Storefront: https://www.amazon.com/shop/cn

Show videos without loading errors

In [11]:
print("Videos that can load...")
vid_page = can_load_page                    # update vid_page with those with no load error
vid_title = can_load_title                  # update vid_title with those with no load error
for i in range(len(vid_title)):
    if vid_title[i] == 'YouTube':           # default error title is 'YouTube'
        vid_title[i] = 'Video_' + str(i+1)  # replace 'YouTube' with Video_1 format
    print(i + 1, vid_title[i])


Videos that can load...
1 Apple Watch Series 9 Review: Slightly Smarter and Faster
2 Apple Watch Series 9 In-Depth Review: Worth Upgrading?
3 Apple Watch Series 9 Review - 6 Months Later
4 Apple Watch Series 9 & Ultra 2: What Are We Waiting For?!
5 Apple Watch Series 9 After 5 Months: Not All It Seems
6 Apple Watch Series 9 — 2 Months After (Long-Term Review)
7 Apple Watch Series 9 After 1 Month: It's Not Even CLOSE!
8 Apple Watch Series 9 One Month Later - Why It Wins
9 Apple Watch Series 9 : Full SCIENTIFIC Review
10 I Hated Smartwatches
11 Apple Watch Series 9 Review: Slightly Smarter and Faster
12 Apple Watch Series 9 In-Depth Review: Worth Upgrading?
13 Apple Watch Series 9 Review - 6 Months Later
14 Apple Watch Series 9 & Ultra 2: What Are We Waiting For?!
15 Apple Watch Series 9 After 5 Months: Not All It Seems
16 Apple Watch Series 9 — 2 Months After (Long-Term Review)
17 Apple Watch Series 9 After 1 Month: It's Not Even CLOSE!
18 Apple Watch Series 9 One Month Later - Why It W

In [12]:
import os

In [13]:
def save_transcript_to_file(video_id, output_folder):
    try:
        # Get the transcript for the video
        transcript = YouTubeTranscriptApi.get_transcript(video_id)

        # Extract the text from the transcript
        text = ''
        for segment in transcript:
            text += segment['text'] + ' '

        # Create the output folder if it doesn't exist
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        # Save the text to a file
        output_file = os.path.join(output_folder, f"{video_id}_transcript.txt")
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(text)

        
        print("Transcript saved successfully!")
    except Exception as e:
        print("An error occurred:", str(e))


for video_id in vid_id:
    save_transcript_to_file(video_id, "transcript_data")
# Example usage
# video_id = '5KfXKQOmqY8'  # Replace 'VIDEO_ID' with the actual ID of the YouTube video
# output_file = 'transcript.txt'

# save_transcript_to_file(video_id, output_file)


Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!
Transcript saved successfully!


Create directory

In [14]:
try:                                              # Create directory named after search terms
    os.makedirs("support/%s" % search_terms)
    print("Directory", search_terms, "created")
except FileExistsError:
    print("Directory", search_terms, "exists")

try:                                              # Create directory to store current search terms
    os.makedirs("support/_current_")
    print("Directory _current_ created")
except FileExistsError:
    print("Directory _current_ exists")


Directory Apple Watch Series 9 Review created
Directory _current_ exists


Save files for future use

In [15]:


pickle.dump(search_terms, open("support/%s/searchTerms.pkl" % search_terms, "wb"))
# pickle.dump(comment_list, open("support/%s/comment_list.pkl" % search_terms, "wb"))
pickle.dump(vid_title, open("support/%s/vid_title.pkl" % search_terms, "wb"))
pickle.dump(vid_page, open("support/%s/vid_page.pkl" % search_terms, "wb"))
pickle.dump(vid_id, open("support/%s/vid_id.pkl" % search_terms, "wb"))


Save files for next step

In [16]:
import shutil

# source = "support/%s/comments.txt" % search_terms
# destination = "support/_current_/comments.txt"
# shutil.copyfile(source, destination)

pickle.dump(search_terms, open("support/_current_/searchTerms.pkl", "wb"))


Clear data for rerun

In [17]:
comment_list.clear()
vid_title.clear()
vid_page.clear()
vid_id.clear()


In [18]:
import os

f = open("keys/openai_key.txt", "r")
key = f.readlines()[0]
f.close()

os.environ["OPENAI_API_KEY"] = key      # LangChain requires API key in environment


Load Private Documents


In [20]:
from langchain_community.document_loaders import DirectoryLoader

from langchain.document_loaders import TextLoader



loader = DirectoryLoader('transcript_data/', glob="./*.txt", loader_cls=TextLoader)

document = loader.load()

Split documents into smaller parts


In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(document)

print("Number of splits in document loaded:", len(splits))


Number of splits in document loaded: 370


Use openAI Embeddings


In [22]:

from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()


Remove 'persist' directory, setup folder


In [23]:
import shutil
import pandas as pd
search_terms = pd.read_pickle("support/_current_/searchTerms.pkl")
print("Processing folder:", search_terms)

try:
    shutil.rmtree('support/%s/persist' % search_terms)       # remove old version
    print("Deleting previous store")
except:
    print("No store found")

persist_directory = 'support/%s/persist' % search_terms     # create new version

files = os.listdir("support/%s" % search_terms)             # list all files
print("Files in folder:", files)


Processing folder: Apple Watch Series 9 Review
No store found
Files in folder: ['searchTerms.pkl', 'vid_id.pkl', 'vid_page.pkl', 'vid_title.pkl']


In [24]:
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(
    documents=splits,                           # target the splits created from the documents loaded
    embedding=embedding,                        # use the OpenAI embedding specified
    persist_directory=persist_directory         # store in the persist directory for future use
)

vectordb.persist()                              # store vectordb

print("Persist Directory created.")
print("Size of Vector Database:", vectordb._collection.count())    # same as the number of splits


Persist Directory created.
Size of Vector Database: 370


Retrieve vectordb


In [25]:
import pandas as pd
search_terms = pd.read_pickle("support/_current_/searchTerms.pkl")

from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

from langchain.vectorstores import Chroma
persist_directory = 'support/%s/persist' % search_terms

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
    )

print("Processing folder:", search_terms)
print("Size of Vector Database", vectordb._collection.count())    # same as before


Processing folder: Apple Watch Series 9 Review
Size of Vector Database 370


In [26]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4", temperature=0)               # gpt model can be changed

In [32]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    # retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 4, "fetch_k": 6}),
    # retriever=vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4}),
    retriever=vectordb.as_retriever(),
    return_source_documents=True
    )


question = "I want you to tell me what the product is, and give me some insights with regard to the following categories: Positively reviewed components, negatively reviewed components, places for improvement, and a general summary. After giving me that information, identify design opportunities and provide me with recommendations. "  # input question


result = qa_chain({"query": "Based on the context provided to you, tell me what the product is. Tell me its good components, bad components, and places for improvement."})

# print("\n--Results with metadata--")
# print(result)
# print("\n--The prompt--")
# print(result["query"])
# print("\n--The sources--")
# print(result["source_documents"])
# print("\n--The final response--")
print(result["result"])

The context provided does not specify the exact product being discussed. However, it seems to be an Apple smartwatch. The good components mentioned include Apple's bold approach to design, challenging the traditional watch aesthetic and opting for a modular design. The product is also praised for its energy efficiency and the company's consideration of the materials, energy, and emissions used in its creation, assembly, packaging, and shipping. The context does not provide any specific bad components or areas for improvement for the product.
